In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import transformers


In [ ]:
train_df=pd.read_csv("../input/contradictory-my-dear-watson/train.csv")
test_df=pd.read_csv("../input/contradictory-my-dear-watson/test.csv")

In [ ]:
max_length=50
epochs=2
batch_size=64

In [ ]:
train_df.shape[0],test_df.shape[0]

In [ ]:
train_df.label.value_counts()

In [ ]:
y_train=tf.keras.utils.to_categorical(train_df.label,num_classes=3)

In [ ]:
class BertSemanticDataGenerator(tf.keras.utils.Sequence):
    """Generates batches of data.

    Args:
        sentence_pairs: Array of premise and hypothesis input sentences.
        labels: Array of labels.
        batch_size: Integer batch size.
        shuffle: boolean, whether to shuffle the data.
        include_targets: boolean, whether to incude the labels.

    Returns:
        Tuples `([input_ids, attention_mask, `token_type_ids], labels)`
        (or just `[input_ids, attention_mask, `token_type_ids]`
         if `include_targets=False`)
    """

    def __init__(
        self,
        sentence_pairs,
        labels,
        batch_size=batch_size,
        shuffle=True,
        include_targets=True,
    ):
        self.sentence_pairs = sentence_pairs
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        # Load our BERT Tokenizer to encode the text.
        # We will use base-base-uncased pretrained model.
        self.tokenizer = transformers.BertTokenizer.from_pretrained(
            "bert-base-multilingual-cased", do_lower_case=True
        )
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch.
        return len(self.sentence_pairs) // self.batch_size

    def __getitem__(self, idx):
        # Retrieves the batch of index.
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence_pairs = self.sentence_pairs[indexes]

        # With BERT tokenizer's batch_encode_plus batch of both the sentences are
        # encoded together and separated by [SEP] token.
        encoded = self.tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True,
            max_length=max_length,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            return_tensors="tf",
        )

        # Convert batch of encoded features to numpy array.
        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        # Set to true if data generator is used for training/validation.
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

    def on_epoch_end(self):
        # Shuffle indexes after each epoch if shuffle is set to True.
        if self.shuffle:
            np.random.RandomState(42).shuffle(self.indexes)

In [ ]:
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # Encoded token ids from BERT tokenizer.
    input_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="input_ids"
    )
    # Attention masks indicates to the model which tokens should be attended to.
    attention_masks = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="attention_masks"
    )
    # Token type ids are binary masks identifying different sequences in the model.
    token_type_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="token_type_ids"
    )
    # Loading pretrained BERT model.
    bert_model = transformers.TFBertModel.from_pretrained("bert-base-multilingual-cased")
    # Freeze the BERT model to reuse the pretrained features without modifying them.
    bert_model.trainable = False

    sequence_output, pooled_output = bert_model(
        input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids
    )
    # Add trainable layers on top of frozen layers to adapt the pretrained features on the new data.
   
    output = tf.keras.layers.Dense(3, activation="softmax")(sequence_output[:,0,:])
    model = tf.keras.models.Model(
        inputs=[input_ids, attention_masks, token_type_ids], outputs=output
    )

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss="categorical_crossentropy",
        metrics=["acc"],
    )


print(f"Strategy: {strategy}")
model.summary()

In [ ]:
train_data=BertSemanticDataGenerator(train_df[["premise","hypothesis"]].values.astype("str"),
                                   y_train,batch_size=batch_size,shuffle=True )


In [ ]:
history=model.fit(train_data,validation_data=None,epochs=epochs,use_multiprocessing=True,workers=-1)

In [ ]:
test_data=BertSemanticDataGenerator(
test_df[["premise","hypothesis"]].values.astype("str"),
labels=None,    
batch_size=batch_size,
shuffle=False)

In [ ]:
def check_similarity(sentence1,sentence2):
    sentence_pairs=np.array([[str(sentence1),str(sentence2)]])
    test_data=BertSemanticDataGenerator(
    sentence_pairs,labels=None,batch_size=1,shuffle=False,include_targets=False)
    proba=model.predict(test_data)[0]
    idx=np.argmax(proba)
    return idx
    

In [ ]:
check_similarity("hello everyone","i love ml")

In [ ]:
sub=pd.read_csv("../input/contradictory-my-dear-watson/sample_submission.csv")

In [ ]:
test_df['label']=0


In [ ]:
test_df.shape[0]

In [ ]:
result=[]
for i in range(5195):
    a=check_similarity(test_df.premise.values[i],test_df.hypothesis.values[i])
    result.append(a)

In [ ]:
result